# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 марта 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 27 марта, -4 балла после 06:00 3 апреля, -6 баллов после 06:00 10 апреля

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы на fit сравнимой со sklearn wine и Speed Dating Data. 
Для этого используем numpy. 

###### Задание 3 (2 балла)
Продемонстрируйте умение работать с Pipeline на данных Speed Dating Data и DecisionTreeClassifier. Нужно в pipeline произвести все необходимые преобразования данных и в конце обучить модель. Задание реализуйте под пунктом Задание 3 (уже написано ниже)

###### Задание 4 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 5 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
from sklearn.model_selection import cross_val_score
%matplotlib inline
%load_ext pycodestyle_magic

In [2]:
# %%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None
                 ):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = np.float32(l_s)
        r_s = np.float32(r_s)
        return np.array(
            (l_s / (l_s + r_s)) *
            (1.0 - (np.array(l_c / l_s, dtype=np.float32) ** 2)
             .sum(axis=-1)).reshape(-1, 1) +
            (r_s / (l_s + r_s)) *
            (1.0 - (np.array(r_c / r_s, dtype=np.float32) ** 2)
             .sum(axis=-1)).reshape(-1, 1), dtype=np.float32
                        )

    def __entropy(self, l_c, l_s, r_c, r_s):
        l_s = np.float32(l_s)
        r_s = np.float32(r_s)

        # чтобы не делать лишних вычислений
        tmp_l = np.array(l_c / l_s, dtype=np.float32)
        tmp_r = np.array(r_c / r_s, dtype=np.float32)
        return -1.0 * (
                (l_s / (l_s + r_s)) * np.nan_to_num(tmp_l * np.log(tmp_l))
                .sum(axis=1).reshape(-1, 1) +
                (r_s / (l_s + r_s)) * np.nan_to_num(tmp_r * np.log(tmp_r))
                .sum(axis=1).reshape(-1, 1)
        )

    def __misclass(self, l_c, l_s, r_c, r_s):
        l_s = np.float32(l_s)
        r_s = np.float32(r_s)
        return (l_s / (l_s + r_s) *
                (1 - np.array(l_c / l_s, dtype=np.float32)
                 .max(axis=1)).reshape(-1, 1) +
                r_s / (l_s + r_s) *
                (1 - np.array(r_c / r_s, dtype=np.float32)
                 .max(axis=1)).reshape(-1, 1)
                )

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:np.sqrt(n_feature)]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:np.log2(n_feature)]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] <= threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):

        # Что делает этот блок кода?

        # сортировка вещественного признака
        sorted_x, sorted_y = self.__sort_samples(x, y)
        # число уникальных классов
        class_number = np.unique(y).shape[0]
        # максимальный номер класса, фикс
        # если в множестве остались только классы, например 1 и 2,
        # то для хранения массива классов нужен
        # массив размера 3, а не 2 (для работы индексации по номеру класса)
        max_class = np.max(y) + 1

        # Что делает этот блок кода?

        # размер отрезанной части записей, фикс
        # нужен для корректного отбрасывания записей при поиске порогов
        # min_samples_split это минимальный размер
        # таблицы записей необходимый для разбиения,
        # это число не подходит для отсечения начала и конца sorted_y
        cut_size = np.int(self.min_samples_split / 2) - 1
        # отделение от начала и конца sorted_y cut_size элементов
        # изменён срез для корректной обработки cut_size = 0
        splitted_sorted_y = sorted_y[cut_size:cut_size + sorted_y.size]
        # поиск индексов sorted_y, на которых происходит смена класса
        # (правые границы областей с одинаковыми классами)
        r_border_ids = np.where(splitted_sorted_y[:-1] !=
                                splitted_sorted_y[1:])[0] + cut_size + 1

        if len(r_border_ids) == 0:
            return float('+inf'), None

        # Что делает этот блок кода?

        # счётчик элементов между правыми границами
        eq_el_count = r_border_ids - np.append([cut_size], r_border_ids[:-1])
        # создание таблицы one hot code для правых границ и классов областей,
        # границами которых они являются
        one_hot_code = np.zeros((r_border_ids.shape[0], max_class))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        # новая таблица, как one_hot_code, но вместо единиц размеры областей
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        # прибавление к первой строке классов
        # начального "хвоста" отсортированного признака
        class_increments[0] = class_increments[0] + np.bincount(
                                          sorted_y[:cut_size],
                                          minlength=max_class
                                          )

        # Что делает этот блок кода?

        # i-я строка этой таблицы - количество элементов каждого класса,
        # находящихся слева r_border_ids[i]
        l_class_count = np.cumsum(class_increments, axis=0)
        # тоже самое, только справа
        r_class_count = np.bincount(sorted_y) - l_class_count
        # размеры частей x слева и справа от r_border_ids[i]
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?

        # рассчёт меры информации
        # при каждом рассматриваемом пороге для предиката
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        # порог предиката, при котором достигается минимум неопределённости
        idx = np.argmin(gs)

        # Что делает этот блок кода?

        # рассчёт оптимального порога для предиката,
        # возврат порога и метрики неопределённости
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id - 1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node
        if node_id == 0:
            self.tree.clear()

        if y.size == 0:
            return
        if np.unique(y).shape[0] == 1:
            self.__init_leaf(x, y, node_id)
            return

        if depth == self.max_depth or \
                np.max(np.bincount(y)) >= \
                np.int64(y.size * self.sufficient_share) or \
                y.size < self.min_samples_split:
            self.__init_leaf(x, y, node_id)
            return

        feature_ids = self.get_feature_ids(x.shape[1])

        classes_numb = np.max(y) + 1
        thresholds = np.zeros((2, x.shape[1]), dtype=np.float32)
        for i in range(x.shape[1]):

            if np.in1d(i, feature_ids):
                tmp, tmp_c = np.unique(x[:, i], return_counts=True)
                if tmp.shape[0] == 2:
                    thresholds[1, i] = (tmp[0] + tmp[1]) / 2
                    l_c = np.bincount(y[np.where(x[:, i] == tmp[0])],
                                      minlength=classes_numb
                                      )
                    r_c = np.bincount(y) - l_c
                    thresholds[0, i] = np.float(
                        self.G_function(l_c, tmp_c[0], r_c, tmp_c[1])
                    )
                else:
                    thresholds[:, i] = np.array(
                        self.__find_threshold(x[:, i], y))

        tmp = np.min(thresholds[0])
        tmp_ids = np.where(thresholds[0] == tmp)[0]
        feature_id = np.random.choice(tmp_ids)
        threshold = thresholds[1, feature_id]

        x_l, x_r, y_l, y_r = self.__div_samples(x, y, feature_id, threshold)

        if y_l.size == 0 or y_r.size == 0:
            self.__init_leaf(x, y, node_id)
            return
        self.__set_importance(y_l, y_r, feature_id)
        self.__init_none_leaf(node_id, feature_id, threshold)
        self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1, feature_id)
        self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1, feature_id)

    def __init_leaf(self, x, y, node_id):
        node = []
        node.append(self.LEAF_TYPE)
        tmp = np.bincount(y)
        tmp_probas = tmp / tmp.sum()
        node.append(np.argmax(tmp))
        node.append(tmp_probas)
        self.tree.update({node_id: node})

    def __init_none_leaf(self, node_id, feature_id, threshold):
        node = []
        node.append(self.NON_LEAF_TYPE)
        node.append(feature_id)
        node.append(threshold)
        self.tree.update({node_id: node})

    def __set_importance(self, y_l, y_r, feature_id):
        classes_numb = np.max(np.array([np.max(np.bincount(y_l)),
                                        np.max(np.bincount(y_r))])) + 1
        self.feature_importances_[feature_id] += \
            y_l.size / (y_l.size + y_r.size) * \
            (1.0 - self.G_function(np.bincount(y_l, minlength=classes_numb),
                                   y_l.size, y_r.size, y_r.size)
             )

    def fit(self, x, y):
        self.feature_importances_ = np.zeros((x.shape[1],), dtype=np.float32)
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] <= threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] <= threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

    def score(self, x, y):
        pred_y = self.predict(x)
        return 1.0 - (np.bincount(np.abs(y - pred_y))[1:]).sum() / y.size

In [9]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [10]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data,
                                                    wine.target,
                                                    test_size=0.1,
                                                    stratify=wine.target)

## Проверка скорости работы на wine

In [11]:
%time clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 2.6 ms, total: 2.6 ms
Wall time: 3.49 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [12]:
%time my_clf.fit(X_train, y_train)

CPU times: user 49.3 ms, sys: 3.74 ms, total: 53 ms
Wall time: 51.8 ms


## Проверка качества работы на wine

In [13]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8857142857142858

In [14]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8857142857142858

In [15]:
# средняя разность score sklearn дерева и score моего дерева
# по 100 выборкам, добавил клетку. Работает несколько секунд
tmp_sum = 0.0
for _ in range(100):
    X_train, X_test, y_train, y_test = train_test_split(wine.data,
                                                        wine.target,
                                                        test_size=0.1,
                                                        stratify=wine.target)
    clf.fit(X_train, y_train)
    my_clf.fit(X_train, y_train)
    score1 = f1_score(y_pred=clf.predict(X_test),
                      y_true=y_test,
                      average='macro')
    score2 = f1_score(y_pred=my_clf.predict(X_test),
                      y_true=y_test,
                      average='macro')
    tmp_sum += score1 - score2
print(tmp_sum / 100)

-0.002617409823292173


## Подготовка данных Speed Dating Data 

In [23]:
# %%pycodestyle
class Cleaner(TransformerMixin):

    def __init__(self):
        self.df = None
        self.X = None
        self.y = None

    def fit(self, df):
        self.df = df
        return self

    def transform(self, df=None):
        if df is None:
            df = self.df
        df = df.iloc[:, :97]
        df = df.drop(['id'], axis=1)
        df = df.drop(['idg'], axis=1)
        df.drop_duplicates(subset=['iid']).gender.value_counts()
        df.drop_duplicates(subset=['iid']).condtn.value_counts()
        df = df.drop(['condtn'], axis=1)
        df = df.drop(['round'], axis=1)
        df = df.drop(['position', 'positin1'], axis=1)
        df = df.drop(['order'], axis=1)
        df = df.drop(['partner'], axis=1)
        df = df.drop(['age_o', 'race_o', 'pf_o_att',
                      'pf_o_sin', 'pf_o_int',
                      'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
                      'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
                      'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o'],
                     axis=1)
        df.drop_duplicates('iid').age.isnull().sum()
        df = df.dropna(subset=['age'])
        df.field_cd.isnull().sum()
        df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
        df = df.drop(['field'], axis=1)
        df = df.drop(['undergra'], axis=1)
        df.loc[:, 'mn_sat'] = \
            df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
        df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
        df.loc[:, 'tuition'] = \
            df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
        df.loc[:, 'tuition'] = df.tuition.fillna(-999)
        df = df.dropna(subset=['imprelig', 'imprace'])
        df = df.drop(['from', 'zipcode'], axis=1)
        df.loc[:, 'income'] = \
            df.loc[:, 'income'].str.replace(',', '').astype(np.float)
        df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
        df = df.dropna(subset=['date'])
        df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
        df = df.drop(['career'], axis=1)
        df = df.drop(['sports', 'tvsports', 'exercise', 'dining',
                      'museums', 'art', 'hiking', 'gaming',
                      'clubbing', 'reading', 'tv', 'theater', 'movies',
                      'concerts', 'music', 'shopping', 'yoga'], axis=1)
        df = df.drop(['expnum'], axis=1)

        feat = ['iid', 'wave', 'attr1_1', 'sinc1_1',
                'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']
        temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
        temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
        df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1',
                                                'intel1_1', 'fun1_1',
                                                'amb1_1', 'shar1_1'
                                                ]].sum(axis=1)
        df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
                   'fun1_1', 'amb1_1', 'shar1_1']] = \
            (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
                        'fun1_1', 'amb1_1', 'shar1_1']].T /
             df.loc[:, 'temp_totalsum'].T).T * 100
        feat = ['iid', 'wave', 'attr2_1', 'sinc2_1',
                'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']
        temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
        temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
        df.loc[:, 'temp_totalsum'] = df.loc[:,
                                            ['attr2_1', 'sinc2_1', 'intel2_1',
                                                'fun2_1', 'amb2_1', 'shar2_1'
                                             ]
                                            ].sum(axis=1)
        df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                   'fun2_1', 'amb2_1', 'shar2_1']] = \
            (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                        'fun2_1', 'amb2_1', 'shar2_1']].T /
             df.loc[:, 'temp_totalsum'].T).T * 100
        df = df.drop(['temp_totalsum'], axis=1)

        for i in [4, 5]:
            feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
                    'intel{}_1'.format(i), 'fun{}_1'.format(i),
                    'amb{}_1'.format(i), 'shar{}_1'.format(i)]

            if i != 4:
                feat.remove('shar{}_1'.format(i))

            df = df.drop(feat, axis=1)

        df = df.drop(['wave'], axis=1)

        # кодирование
        tmp = pd.get_dummies(df.loc[:, 'career_c'],
                             prefix='career_class',
                             prefix_sep='=')
        df = pd.concat([df, tmp], axis=1)
        tmp = pd.get_dummies(df.loc[:, 'race'], prefix='race', prefix_sep='=')
        df = pd.concat([df, tmp], axis=1)

        df_male = df.query('gender == 1') \
            .drop_duplicates(subset=['iid', 'pid']) \
            .drop(['gender'], axis=1) \
            .dropna()
        df_female = df.query('gender == 0').drop_duplicates(subset=['iid']) \
            .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1) \
            .dropna()

        df_female.columns = df_female.columns + '_f'
        df_female = df_female.drop(['pid_f'], axis=1)
        df_pair = df_male.join(df_female.set_index('iid_f'),
                               on='pid',
                               how='inner')
        df_pair = df_pair.drop(['iid', 'pid'], axis=1)
        return df_pair


df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv',
                 encoding='cp1251')
cleaner = Cleaner()
cleaner.fit(df)
df_pair = cleaner.transform()
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,
                                                    random_state=123)
X.shape

(3999, 108)

## Проверка скорости работы на Speed Dating Data 

In [24]:
%time clf.fit(X_train, y_train)

CPU times: user 108 ms, sys: 0 ns, total: 108 ms
Wall time: 106 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [25]:
%time my_clf.fit(X_train, y_train)

CPU times: user 1.59 s, sys: 7.75 ms, total: 1.59 s
Wall time: 1.59 s


## Проверка качества работы на Speed Dating Data

In [26]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5718929732433109

In [27]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5

In [28]:
# то же самое, что и с wine_data, но с 10 выборками
tmp_sum = 0.0
for _ in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X,
                                                        y,
                                                        test_size=0.1,
                                                        random_state=123)
    clf.fit(X_train, y_train)
    my_clf.fit(X_train, y_train)
    score1 = f1_score(y_pred=clf.predict(X_test),
                      y_true=y_test, average='macro')
    score2 = f1_score(y_pred=my_clf.predict(X_test),
                      y_true=y_test, average='macro')
    tmp_sum += score1 - score2
print(tmp_sum / 10)

0.07175771968494027


## Задание 3

In [29]:
df = pd.read_csv('speed-dating-experiment/Speed Dating Data.csv',
                 encoding='cp1251'
                 )

ppln_clear = Pipeline([
    ('clean', Cleaner()),
])
ppln_predict = Pipeline([
    ('clf', DecisionTreeClassifier(min_samples_split=2))
])


df_pair = ppln_clear.transform(df)
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values

cross_val_score(ppln_predict, X, y).mean()

0.6849212303075768

## Задание 4

In [30]:
df_pair = ppln_clear.transform(df)
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.1, random_state=123)

my_clf = MyDecisionTreeClassifier(min_samples_split=2)
my_clf.fit(X_train, y_train)
clf = DecisionTreeClassifier(min_samples_split=2)
clf.fit(X_train, y_train)
print(np.argsort(my_clf.feature_importances_)[:-10:-1])
print(np.argsort(clf.feature_importances_)[:-10:-1])

[18 73 76 56 55 72 74  6  2]
[ 0 62 72 55 18 67 71 70  7]


## Задание 5

In [31]:
df_pair = ppln_clear.transform(df)
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.1, random_state=123)
parameters = {'n_estimators': np.arange(5, 21),
              'min_samples_split': np.arange(2, 8)}
clf = GridSearchCV(RandomForestClassifier(), parameters)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.8375